# Advanced Tutorial

Welcome to the advanced iminuit tutorial. If you are new to iminuit, you should go through the basic tutorial first. This short tutorial deals with some corner cases where the Python introspection cannot detect the parameter names automatically, but where you can work around that. It also shows how to speed up the computation of the likelihood with Cython.

In [1]:
# setup of the notebook
%load_ext Cython
%pylab inline
from iminuit import Minuit, describe

Populating the interactive namespace from numpy and matplotlib


## Faster fits with Cython

Usually most of the computation time is spend in the fit function when you run iminuit. If you want more speed with minimal code change, Cython is one of the ways. We will only give a quick introduction to cython. For a hard core cython see hard-core-tutorial.ipynb.

In [2]:
%%cython --force
cimport cython
import numpy as np
cimport numpy as np

@cython.embedsignature(True)  # put function signature in pydoc so `describe` can extract it
def cython_f(double x,double y,double z):
    return (x - 1.) ** 2 + (y - 2.) ** 2 + (z - 3.) ** 2 + 1.

In [3]:
m = Minuit(cython_f, pedantic=False, errordef=1)
fmin, param = m.migrad()

FCN = 1.0 
 TOTAL NCALL = 36 
 NCALLS = 36 
 
 
 EDM = 7.96484604487e-23 
 GOAL EDM = 1e-05 
 
 UP = 1.0 
 
 

 
 
 Valid 
 Valid Param 
 Accurate Covar 
 PosDef 
 Made PosDef 
 
 
 True 
 True 
 True 
 True 
 False 
 
 
 Hesse Fail 
 HasCov 
 Above EDM 
 
 Reach calllim 
 
 
 False 
 True 
 False 
 
 False

Name 
 Value 
 Hesse Error 
 Minos Error- 
 Minos Error+ 
 Limit- 
 Limit+ 
 Fixed? 
 
 
 0 
 x 
 1 
 1 
 
 
 
 
 No 
 
 
 1 
 y 
 2 
 1 
 
 
 
 
 No 
 
 
 2 
 z 
 3 
 1 
 
 
 
 
 No

## Generic least squares function with faked function signature

We have seen in the basic tutorial how to make a least-squares function with an explicit signature that iminuit could read to find the parameter names automatically. Part of the structure of a least-squares function is always the same. What changes is the model that predicts the y-values and its parameters. So, let's try to make a generic least-squares class that works with iminuit.

In [4]:
class LeastSquares:
    def __init__(self, model, x, y):
        self.model = model  # model predicts y for given x
        self.x = array(x)
        self.y = array(y)

    def __call__(self, *par):  # par are a variable number of model parameters
        ym = self.model(self.x, *par)
        chi2 = sum((self.y - ym)**2)
        return chi2

In [5]:
def line(x, a, b):  # simple straight line model with explicit parameters
    return a + b * x

lsq = LeastSquares(line,
                   [1, 2, 3, 4, 5],
                   [2, 4, 6, 8, 10])

try:
    describe(lsq)  # this raises a TypeError, because signature cannot be read
except TypeError as e:
    print(e)

Unable to obtain function signature


What happened? `describe` only sees the method `LeastSquares.__call__(self, *par)`, but knowns nothing about the signature of `line(x, a, b)`. We can fix this by faking the function signature of the `LeastSquares` class.

In [6]:
from iminuit.util import make_func_code
# get the args from line and strip 'x'
par_names = describe(line)[1:]
lsq.func_code = make_func_code(par_names)

# now we get the right answer
describe(lsq)

['a', 'b']

In [7]:
m = Minuit(lsq, pedantic=False)
fmin, param = m.migrad()

FCN = 6.48916980635e-26 
 TOTAL NCALL = 32 
 NCALLS = 32 
 
 
 EDM = 6.48992044656e-26 
 GOAL EDM = 1e-05 
 
 UP = 1.0 
 
 

 
 
 Valid 
 Valid Param 
 Accurate Covar 
 PosDef 
 Made PosDef 
 
 
 True 
 True 
 True 
 True 
 False 
 
 
 Hesse Fail 
 HasCov 
 Above EDM 
 
 Reach calllim 
 
 
 False 
 True 
 False 
 
 False

Name 
 Value 
 Hesse Error 
 Minos Error- 
 Minos Error+ 
 Limit- 
 Limit+ 
 Fixed? 
 
 
 0 
 a 
 3.37508e-14 
 1.04881 
 
 
 
 
 No 
 
 
 1 
 b 
 2 
 0.316228 
 
 
 
 
 No

We put the code for this trick into the init function of `LeastSquares` and obtain a generic least-squares class which works with iminuit.

In [8]:
class LeastSquares:  # override the class with a better one
    def __init__(self, model, x, y):
        self.model = model  # model predicts y for given x
        self.x = array(x)
        self.y = array(y)
        self.func_code = make_func_code(describe(self.model)[1:])

    def __call__(self, *par):  # par are a variable number of model parameters
        ym = self.model(self.x, *par)
        chi2 = sum((self.y - ym)**2)
        return chi2

In [9]:
lsq = LeastSquares(line,
                   [1, 2, 3, 4, 5],
                   [2, 4, 6, 8, 10])
describe(lsq)  # works!

['a', 'b']

## Last Resort: Forcing function signature

built-in functions normally do not have a function signature. Functions from cython and swig also do not have one. Python introspection fails and we have to force function signature.

In [10]:
%%cython
# example with absolutely no signature, generated with cython
# note: you can add a signature with the decorate @cython.embedsignature(True) as shown above
def nosig_f(x,y):
    return x**2+(y-4)**2

In [11]:
try:
    describe(nosig_f)  # raises error
except TypeError as e:
    print(e)

Unable to obtain function signature


In [12]:
# you can always force parameters, like so
m = Minuit(nosig_f, forced_parameters=('x','y'), pedantic=False)

In [13]:
m.migrad();

FCN = 9.31236296332e-23 
 TOTAL NCALL = 24 
 NCALLS = 24 
 
 
 EDM = 9.31236296802e-23 
 GOAL EDM = 1e-05 
 
 UP = 1.0 
 
 

 
 
 Valid 
 Valid Param 
 Accurate Covar 
 PosDef 
 Made PosDef 
 
 
 True 
 True 
 True 
 True 
 False 
 
 
 Hesse Fail 
 HasCov 
 Above EDM 
 
 Reach calllim 
 
 
 False 
 True 
 False 
 
 False

Name 
 Value 
 Hesse Error 
 Minos Error- 
 Minos Error+ 
 Limit- 
 Limit+ 
 Fixed? 
 
 
 0 
 x 
 0 
 1 
 
 
 
 
 No 
 
 
 1 
 y 
 4 
 1 
 
 
 
 
 No